In [1]:
import nltk

In [2]:
import re

In [3]:
import string

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
from sklearn.naive_bayes import MultinomialNB

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

In [12]:
def segmentation(text):
    pun = nltk.data.load('tokenizers/punkt/english.pickle')
    return pun.tokenize(text)

In [13]:
def feature_extraction(txtlst):
    vec = TfidfVectorizer().fit(txtlst)
    return vec.transform(txtlst)

In [14]:
def remove_newLine(text):
    return text.replace('\n','')

In [15]:
def remove_html(text):
    return re.sub('<.*?>','',text)

In [16]:
def remove_URL(text):
    return re.sub('https?:\/\/[^\s]+', '', text)

In [17]:
def split_punc(text):
    puncs = '!"#$%&()*+,./:;<=>?@[\\]^_`{|}~'
    abbList = [line.rstrip('\n') for line in open(r'/home/samaaessa/Desktop/NLP tasks/project/abbrev.english')]
    return ' '.join([w.strip(puncs) for w in text.split() if w not in abbList])

In [18]:
def clitics(text):
    clitics = [line.rstrip('\n') for line in open(r'/home/samaaessa/Desktop/NLP tasks/project/clitics')]
    newtxt = ''
    nWord = ''
    for word in text.split():
        for clitic in clitics:
            if clitic in word:
                nWord = word.replace(clitic," "+clitic)+" "
                break
            else:
                nWord = word+" "
        newtxt += nWord
    return newtxt

In [19]:
def remove_stopwords(text):
    stopwords = [line.rstrip('\n') for line in open(r'/home/samaaessa/Desktop/NLP tasks/project/StopWords')]
    return ' '.join([w for w in text.split() if w not in stopwords])

In [20]:
def lowerCase(text):
    return text.lower()

In [21]:
def preprocess(text):
    text = remove_newLine(text)
    text = remove_html(text)
    text = remove_URL(text)
    text = split_punc(text)
    text = clitics(text)
    text = lowerCase(text)
    text = remove_stopwords(text)
    return text

In [22]:
txt = open('dataSet.txt','r').read()

In [23]:
df = pd.read_table('dataSet.txt',sep='\t',header=None,names=['label','sents'])

In [24]:
for i in range(len(df)):
    df['sents'][i] = preprocess(df['sents'][i])

/home/samaaessa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
#print([i for i in range(153) if df['sents'][i] is np.nan])
counts = feature_extraction(df['sents'])

In [26]:
counts  = feature_extraction(df['sents'])

In [27]:
x_train,x_test,y_train,y_test =train_test_split(df['sents'],df['label'],test_size = 0.1,random_state = 69)
x_train_feature,x_test_feature,y_train,y_test =train_test_split(counts,df['label'],test_size = 0.1,random_state = 69)

In [28]:
model = MultinomialNB().fit(x_train_feature,y_train)

In [29]:
predicted = model.predict(x_test_feature)

In [30]:
print('Predicted class:',predicted)
print('True class     :',y_test.values)

Predicted class: [0 0 1 0 1 1 1 0 0 0 0 0 1 1 1]
True class     : [0 0 0 0 0 1 1 0 0 0 0 1 1 1 0]


In [49]:
print('Predicted\tClass\tSentance')
print('--------------------------------------------')
for i in range(len(predicted)):
    print(predicted[i],'\t\t',y_test.values[i],'\t\t',x_test.values[i])

Predicted	Class	Sentance
--------------------------------------------
0 		 0 		
0 		 0 		
1 		 0 		
0 		 0 		
1 		 0 		
1 		 1 		
1 		 1 		
0 		 0 		
0 		 0 		
0 		 0 		
0 		 0 		
0 		 1 		
1 		 1 		
1 		 1 		
1 		 0 		


In [32]:
print(np.mean(predicted == y_test))

0.7333333333333333


In [33]:
 tn, fp, fn, tp = confusion_matrix(y_test,predicted).ravel()

In [34]:
accuracy = (tn + tp)/(tn+tp+fn+fp)

In [35]:
accuracy

0.7333333333333333